In [6]:
# 📓 Batch 4: Generate Engineered/Derived Features from Real Data

import pandas as pd
import numpy as np

# Load real FRED signals (from Batch 1)
fred_df = pd.read_csv("Batch1_fred_signals.csv", parse_dates=["date"], index_col="date")

# Ensure monthly frequency
fred_df = fred_df.resample("M").mean().dropna()

# --- Derived Signals ---
df = pd.DataFrame(index=fred_df.index)

# 🟢 Proxy for nowcast GDP: centered rolling mean of GDP growth
df["nowcast_gdp"] = fred_df["gdp_growth"].rolling(window=3, center=True).mean()

# 🟢 Recession model score: weighted normalized combination of high unemployment & negative GDP growth
score = (
    (fred_df["unemployment_rate"] - fred_df["unemployment_rate"].mean()) / fred_df["unemployment_rate"].std()
    - (fred_df["gdp_growth"] - fred_df["gdp_growth"].mean()) / fred_df["gdp_growth"].std()
)
df["recession_model_score"] = (score - score.min()) / (score.max() - score.min())

# 🟢 Economic surprise index: rolling z-score change in CPI inflation
inflation_diff = fred_df["cpi_inflation"].diff()
rolling_std = inflation_diff.rolling(window=6).std()
df["economic_surprise_index"] = inflation_diff / rolling_std

# 🟢 Liquidity crunch index: VIX spike + drop in consumer sentiment
vix_z = (fred_df["vix_index"] - fred_df["vix_index"].mean()) / fred_df["vix_index"].std()
sentiment_z = (fred_df["consumer_sentiment_index"] - fred_df["consumer_sentiment_index"].mean()) / fred_df["consumer_sentiment_index"].std()
df["liquidity_crunch_index"] = vix_z - sentiment_z

# 🟢 Tail risk score: large jumps in VIX (standardized)
vix_change = fred_df["vix_index"].pct_change()
df["tail_risk_score"] = (vix_change.rolling(3).std()) * 100

# 🟢 Macro risk factor: principal component proxy (simple combination here)
df["macro_risk_factor_1"] = (
    0.3 * fred_df["gdp_growth"].fillna(0) -
    0.5 * fred_df["unemployment_rate"].fillna(0) +
    0.2 * fred_df["vix_index"].fillna(0)
)

# --- Output ---
df = df.dropna()
df.index.name = "date"
df.to_csv("Batch4_engineered_features.csv")
print("✅ Saved Batch4_engineered_features.csv using real data")

✅ Saved Batch4_engineered_features.csv using real data


/var/folders/bq/_yk5zsn90t11g7lt1yn4v9w40000gn/T/ipykernel_11544/4190769383.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fred_df = fred_df.resample("M").mean().dropna()
